### Data Aggregation
#### the data aggregation procedure described in the project, followed by the inference of values
#### only applicable to a databse with all five properties (ZT, thermal conductivity, electrical conductivity, Seebeck coefficient, PF)

In [3]:
import pandas as pd
import numpy as np
from functools import reduce
import math
import json
import statistics
import os

In [1152]:
name_prefix = ""

In [1153]:
# expects the 'intermediary' version of a database, obtained by processing 
# an extracted database with 5 different thermoelectric-materials properties present,
# and setting save_intermediary = True in the "data_cleaning.ipynb" notebook.

intermediary_database_path = "full_intermediary_database.csv"

if os.path.exists(intermediary_database_path):
    dd = pd.read_csv(intermediary_database_path)
else:
    print("Expects an intermediary database, from data cleaning.")
    raise FileNotFoundError

In [1154]:
if dd.normalised_model.nunique() != 5:
    print("Expected 5 different property models.")
    raise Exception

### Aggregating between thermal conductivity types first (total, electronic, and lattice)

In [1155]:
therm = dd[dd.normalised_model == "ThermCond"].copy()

In [1156]:
tot = therm[therm.type == 'total'].copy()
ele = therm[therm.type == 'electronic'].copy()
lat = therm[therm.type == 'lattice'].copy()

In [1157]:
ele.rename({'normalised_avg' : 'EleThermCond_value'}, axis=1, inplace = True)
ele.drop(columns=['normalised_model','type', 'specifier'], inplace = True)
lat.rename({'normalised_avg' : 'LatThermCond_value'}, axis=1, inplace = True)
lat.drop(columns=['normalised_model','type'], inplace = True)
tot.rename({'normalised_avg' : 'ThermCond_value'}, axis=1, inplace = True)
tot.drop(columns=['normalised_model','type'], inplace = True)

In [1158]:
# use inner merging to get only the entries which have both an electronic and lattice component
ele_lat = pd.merge(ele, lat, on=['first_name','normalised_temp_avg','filename',
                                'parser', 'oa', 'publisher', 'editing', 'pressure', 'error', 'process',
                                'labels', 'direction_of_measurement'], how='inner')

In [1159]:
def add_ele_and_lat(df):
    return (df.EleThermCond_value + df.LatThermCond_value)


In [1160]:
if not ele_lat.empty:
    ele_lat['SUMThermCond_value'] = ele_lat.apply(add_ele_and_lat, axis=1)
else:
    ele_lat['SUMThermCond_value'] = None


In [1161]:
tot_th = therm[therm.type == 'total']

In [1162]:
tot_th = tot_th.drop(['type','specifier'], axis=1)

In [1163]:
cols = ['first_name',
       'normalised_avg', 'normalised_temp_values', 'parser', 'filename', 'oa', 'yop', 'publisher',
       'editing', 'pressure', 'error', 'process', 'labels', 'direction_of_measurement']

#### Making changes to include electronic (k_e) and lattice (k_p) as well

In [1164]:
tot_th = therm[therm.type == 'total'][cols].rename({"normalised_avg":"k"}, axis=1).copy()
e_th = therm[therm.type == 'electronic'][cols].rename({"normalised_avg":"k_e"}, axis=1).copy()
p_th = therm[therm.type == 'lattice'][cols].rename({"normalised_avg":"k_p"}, axis=1).copy()
zt = dd[dd.normalised_model == 'ZT'][cols].rename({"normalised_avg":"ZT"}, axis=1).copy()
pf = dd[dd.normalised_model == 'PF'][cols].rename({"normalised_avg":"PF"}, axis=1).copy()
s = dd[dd.normalised_model == 'Seebeck'][cols].rename({"normalised_avg":"S"}, axis=1).copy()
el = dd[dd.normalised_model == 'Conductivity'][cols].rename({"normalised_avg":"s"}, axis=1).copy()

In [1165]:
dfs = [tot_th, e_th, p_th, pf, s, el, zt]

In [1166]:
merge_columns = [
 'first_name',
 'normalised_temp_values',
 'filename',
 'oa', 'yop',
 'parser',
 'publisher',
 'editing', 'pressure', 'process', 'labels', 'direction_of_measurement']

In [1167]:
def collect(s):
    """used to collect multiple values for same merged columns"""
    l = []
    for x in s:
        if has(x):
            l.append(x)
    return l

In [1168]:
dfs_copy = dfs.copy()
dfs_collect = []

In [1169]:
# for brevity. Return False for both [] and np.nan. Unfortunately bool(np.nan) evaluates to True
def has(x):
    if x == []:
        return False
    return not (x is np.nan)


In [1170]:
dfs_mods = "k k_e k_p ZT PF S s".split()

# collect mutliple values for same merge columns (e.g. same compound, same temp, diff value averages)
for dfc in dfs_copy:
    df_mod = [m for m in dfs_mods if m in dfc.columns][0]
    dfs_collect.append(dfc.groupby(merge_columns, dropna = False)[df_mod].apply(collect).reset_index())

# Aggregating

In [1171]:
# merges the databases in pairs, on common compound, labels, filename, and temperature values
# if a value does not exist, it is populated by NaN
db = reduce(lambda df_left,df_right: pd.merge(df_left, df_right, how='outer',
                                              on=merge_columns), dfs_collect)

# NB If there were multiple values, then they are chosen indiscriminantly, addressed this via dfs_collect

In [1172]:
model_columns = ['k', 'k_e', 'k_p', 'PF', 'S', 's', 'ZT']
def count_nans(df):
    """get the total number of nans for the columsn we care about"""
    return df[model_columns].isnull().sum().sum()

In [1173]:
length = 0
for dfc in dfs_collect:
    length += len(dfc)
print(length)

# sanity check two numbers must agree (the rows before aggregate, and the non nan entries in the aggregate db)
db[model_columns].size - count_nans(db)
# db[model_columns].notna().sum().sum()  # otherwise

20819


20819

In [1174]:
def count_quantities(df):
    """count how many of the 5 thermoelectric properties are found in each row"""
    c=0
    if has(df.ZT):
        c+=1
    if has(df.S):
        c+=1
    if has(df.s):
        c+=1
    if has(df.k):  # ignoring k_e and k_p
        c+=1
    if has(df.PF):
        c+=1
    return c

In [1175]:
db['counts'] = db["ZT S s k PF".split()].apply(count_quantities, axis=1)
# NB count is a just column that was added, value_counts is a standard pandas function

In [1176]:
# square Seebeck coefficient
db.S = db.S.apply(lambda l: [x**2.0 for x in l] if (l is not np.nan) else np.nan)

In [1177]:
# rename Seebeck column
db = db.rename({'S': 'S_2'},axis=1)

In [1178]:
db = db[['first_name', 'normalised_temp_values',
    'ZT', 'PF', 'S_2', 's', 'k', 'k_e', 'k_p',
    'counts','parser', 'filename', 'publisher', 'oa', 'yop',
    'editing', 'pressure', 'process', 'labels', 'direction_of_measurement']].copy()  # set order. Maybe should sort as well?

In [1179]:
def make_number_list2(x):
    try:
        x_list = x[1:-1].split(',')

        return [float(n) for n in x_list]
    except:
        return np.nan

In [1180]:
db.normalised_temp_values = db.normalised_temp_values.apply(make_number_list2).copy()

### Keep single values for comparison and inference. (avg)

In [1181]:
# Although not necessary, having a unique value for each property at each data record, facilitates inference
# and comparison between the aggregated data. There are several options, such as taking the average, mode, median
# or other options, when there are multiple values present. This procedure takes the average of values.

In [1182]:
# get averages of values (which are already averages) to perform comparison. It's simple and somewhat reasonable

# to clarify: some extractions return a range, which has been averaged
# then we got those averages, and where the merged columns agreed, added them into single records, in a list
# i.e. we collected different extractions with diff average values, for the same things, and added them into a list
# and now we get the average of the values in that list.

In [1183]:
mod_names = "ZT PF k S_2 s k_e k_p".split() 

In [1184]:
for m in mod_names + ["normalised_temp_values"]:
    db[m] = db[m].apply(lambda l: statistics.mean(l) if isinstance(l, list) else np.nan)
    

## Inference

In [1185]:
# This code merges the 5 normalised databases (on common compound, doi, and temperature) and then uses
# the interdependency between the quantities, to infer new values.

In [1186]:
# bcreate new column to hold the steps of the inference and comparison procedure
db['inference'] = ''

In [1187]:
# for brevity. A bit more flexible than notna because it doesn't crash for conversions (?)
def has(x):
    return not math.isnan(x)

In [1188]:
# inference (updates original column) if unpopulated

def get_PF_from_2(df):
    # ZT and k calculation
    if has(df.ZT) and has(df.k):
        calc1 = df.ZT * df.k / df.normalised_temp_values
        if not(has(df.PF)):
            df.PF = calc1
            df.inference += '[ZT/kT -> PF]'
            
    # S^2 and s calculation
    elif has(df.S_2) and has(df.s):
        calc2 = df.S_2 * df.s
        if not has(df.PF):
            df.PF = calc2
            df.inference += '[S^2*s -> PF]'
    return df

In [1189]:
db = db.apply(get_PF_from_2, axis = 1)

In [1190]:
# repeat similar procedure for the rest of the quantities
def get_S2(df):
    # PF and s
    if has(df.PF) and has(df.s):
        calc1 = df.PF / df.s
        if not(has(df.S_2)):
            df.S_2 = calc1
            df.inference += '[PF/s -> S^2]'
    return df
    # S^2 = ZT * k / (s * T) is skipped because that would have given a PF in first step via PF = ZT * k / T

In [1191]:
db = db.apply(get_S2, axis=1)

In [1192]:
def get_s(df):    
    # PF and S^2
    if has(df.PF) and has(df.S_2):
        calc1 = df.PF / df.S_2
        if not(has(df.s)):
            df.s = calc1
            df.inference += '[PF/S^2 -> s]'
    return df
    # s = ZT * k / (s^2 * T) is skipped because that would have given a PF in first step via PF = ZT * k / T

In [1193]:
db = db.apply(get_s, axis=1)

In [1194]:
def get_ZT(df):
    # PF and k
    if has(df.PF) and has(df.k):
        calc1 = df.PF * df.normalised_temp_values / df.k
        if not(has(df.ZT)):
            df.ZT = calc1
            df.inference += '[PF*T/k -> ZT]'
    return df
    # ZT = S^2 * s * T / k is skipped because that would have given a PF in first step via PF = S^2 * s 

In [1195]:
db = db.apply(get_ZT, axis=1)

In [1196]:
def get_k(df):
    # PF and ZT
    if has(df.PF) and has(df.ZT):
        calc1 = df.PF * df.normalised_temp_values / df.ZT
        if not(has(df.k)):
            df.k = calc1
            df.inference += '[PF*T/ZT -> k]'
    return df
    # k = S^2 * s * T / ZT is skipped because that would have given a PF in first step via PF = S^2 * s 

In [1197]:
db = db.apply(get_k, axis=1)

In [1198]:
# these are the final logistics of inference
results = db.inference.value_counts()

In [1199]:
mod_names = "ZT s S_2 PF k".split()

In [1200]:
db.rename({"counts": "original_counts"}, axis=1, inplace = True)

In [1201]:
def count_aggregated_quantities(df):
    """same as counta_quantities, but with S_2 for Seebeck coefficient squared"""
    c=0
    if has(df.ZT):
        c+=1
    if has(df.S_2):
        c+=1
    if has(df.s):
        c+=1
    if has(df.k):
        c+=1
    if has(df.PF):
        c+=1
    return c

In [1202]:
db.insert(db.columns.get_loc("original_counts") + 1, "new_counts", db.apply(count_aggregated_quantities, axis=1))

### formatting aggregate database with inference

In [1203]:
df_ordered_columns = ['first_name',
 'normalised_temp_values',
 'ZT',
 'PF',
 'S_2',
 's',
 'k',
 'k_e',
 'k_p',
 'original_counts',
 'new_counts',
 'filename',
 'publisher',
 'oa',
 'yop',
 'editing',
 'pressure',
 'process',
 'labels',
 'direction_of_measurement',
 'inference']

In [1204]:
db = db[db_ordered_columns].copy()

In [1205]:
renaming_dict = {
 'first_name': 'Name',
 'labels': 'Label',
 'editing': 'Editing',
 'normalised_model': 'Model',
 'type': 'Model_Type',
 'specifier': 'Specifier',
 'normalised_values': 'Value',
 'normalised_units': 'Units',
 'normalised_temp_values': 'Temperature_Value',
 'normalised_temp_units': 'Temperature_Units',
 'normalised_avg': 'Value_Average',
 'normalised_temp_avg': 'Temperature_Average',
 'pressure': 'Pressure',
 'process': 'Process',
 'direction_of_measurement': 'Direction_of_Measurement',
 'filename': 'DOI',
 'title': 'Title',
 'oa': 'Access_Type',
 'publisher': 'Publisher',
 'yop': 'Publication_Year',
 'S_2': 'S^2',
 'original_counts': 'Original_Counts',
 'new_counts': 'New_Counts',
 'inference': 'Inference'}

In [1206]:
db.rename(renaming_dict, axis=1, inplace=True)
db.head(3)

,Name,Temperature_Value,ZT,PF,S^2,s,k,k_e,k_p,Original_Counts,...,DOI,Publisher,Access_Type,Publication_Year,Editing,Pressure,Process,Label,Direction_of_Measurement,Inference
0,% SnS : Bi2Te3 nanocomposite,295.0,NaN,NaN,NaN,NaN,0.680,NaN,NaN,1,...,10.1016-j.jallcom.2020.156233.xml,Elsevier,no,NaN,NaN,NaN,NaN,NaN,NaN,
1,"(Bi,Sb)2-xSnxSe3–4xCl4x",773.0,0.61,0.0004,NaN,NaN,0.475,NaN,NaN,3,...,10.1016-j.mssp.2019.01.021.xml,Elsevier,no,NaN,NaN,NaN,NaN,NaN,NaN,
2,(Ce0.2Zr0.2Hf0.2Sn0.2Ti0.2)O2,295.0,NaN,NaN,NaN,NaN,1.280,NaN,NaN,1,...,10.1039-c9ta05698j.html,RSC,no,NaN,NaN,NaN,NaN,NaN,NaN,


In [1207]:
def change_access_type_entries(x):
    return "open" if x == "yes" else "payment"

db.Access_Type = db.Access_Type.apply(change_access_type_entries)

In [1208]:
db.Access_Type = db.Access_Type.apply(change_access_type_entries)

In [1209]:
db.head(3)

,Name,Temperature_Value,ZT,PF,S^2,s,k,k_e,k_p,Original_Counts,...,DOI,Publisher,Access_Type,Publication_Year,Editing,Pressure,Process,Label,Direction_of_Measurement,Inference
0,% SnS : Bi2Te3 nanocomposite,295.0,NaN,NaN,NaN,NaN,0.680,NaN,NaN,1,...,10.1016-j.jallcom.2020.156233.xml,Elsevier,payment,NaN,NaN,NaN,NaN,NaN,NaN,
1,"(Bi,Sb)2-xSnxSe3–4xCl4x",773.0,0.61,0.0004,NaN,NaN,0.475,NaN,NaN,3,...,10.1016-j.mssp.2019.01.021.xml,Elsevier,payment,NaN,NaN,NaN,NaN,NaN,NaN,
2,(Ce0.2Zr0.2Hf0.2Sn0.2Ti0.2)O2,295.0,NaN,NaN,NaN,NaN,1.280,NaN,NaN,1,...,10.1039-c9ta05698j.html,RSC,payment,NaN,NaN,NaN,NaN,NaN,NaN,


#### Adding authors and journal from chemataextractor's metadata

In [4]:
meta_path = os.path.join(os.getcwd(), 'resources', 'metadata_dict.json')

In [5]:
with open(meta_path, "rb") as handle:
    meta_dict = json.load(handle)

In [2]:
db.head()

NameError: name 'db' is not defined

In [1213]:
def meta_to_df(df):
    doi = df.DOI
    try:
        authors = meta_dict[doi]['authors']
    except KeyError:
        authors = np.nan
    try:
        journal = meta_dict[doi]["journal"]
    except KeyError:
        journal = np.nan

        
    df.authors = authors
    df.journal = journal
    
    return df

In [1214]:
for c in "authors journal".split():
    db[c] = ""

In [1215]:
db = db.apply(meta_to_df, axis=1)

In [1216]:
# restore the DOI from the filename format
def return_doi_to_original_form(d):
    d = list(d.rsplit('.',1)[0]) # split and index to skip file extension, transform to list
    d[7] = "/"  # replace affected hyphen with original slash
    return "".join(d) # join and return

In [1217]:
db.DOI = db.DOI.apply(return_doi_to_original_form)

In [1218]:
db.rename({s: s.capitalize() for s in "authors journal".split()}, axis=1, inplace = True)

In [1219]:
db_final_order = [
    'Name',
    'Temperature_Value',
    'ZT', 'PF', 'S^2', 's', 'k',
    'Original_Counts', 'New_Counts', 'Inference',
    'Editing', 'Pressure', 'Process', 'Label', 'Direction_of_Measurement',
    'DOI', 'Publisher', 'Access_Type', 'Publication_Year', 'Authors', 'Journal']

In [1220]:
print([i for i in db_final_order if i not in db.columns])
print([c for c in db.columns if c not in db_final_order])

[]
['k_e', 'k_p']


In [1221]:
db = db[db_final_order]

In [1222]:
database_name = "aggregated_database.csv"
save_name = name_prefix + database_name

if not os.path.exists(save_name):
    db.to_csv(save_name, index=False)
    print(f"Saved {save_name}")
else:
    save = input(f"'{save_name}' already exists, would you like to overwrite? [Y/n]")
    if save == "Y":
        db.to_csv(save_name, index=False)
        print(f"Overwrote {save_name}")
    else:
        print("Skipping save.")

'aggregated_database.csv' already exists, would you like to overwrite? [Y/n]Y
Overwrote aggregated_database.csv
